In [68]:
# Importing libraries
import math
import numpy as np
import matplotlib.pyplot as plt

In [69]:
### function for counting number of combinations to be used as modifiers

def count_combs(n):
# def combi(n:int, r:int):
    result = []
    for i in range(n+1):
        result.append(math.comb(n,i))
    
    return result

In [70]:
### setting up the PVP envrionment. -> how many VS how many ?

def create_pvp_env(n1, n2):
    result = []
    result.append(count_combs(n1-1))    ### minus ME
    result.append(count_combs(n2))
    return result

In [71]:
# my_team_combs = count_combs(5)

In [72]:
# my_team_combs

In [73]:
# op_team_combs = count_combs(5)

In [74]:
# op_team_combs

In [75]:
create_pvp_env(5,5)

[[1, 4, 6, 4, 1], [1, 5, 10, 10, 5, 1]]

In [76]:
my_heads_prob = 0.57
ally_heads_prob = 0.54
enemy_heads_prob = 0.54

In [77]:
### team probability, dependent on number of combination counts and percentile

def create_team_probs_dict(n_my_team, n_op_team, my_heads_prob, ally_heads_prob, enemy_heads_prob):
    result = []
    
    ### prob of ally team's tails
    ally_tails_prob = 1 - ally_heads_prob
    ### prob of enemy team's tails
    enemy_tails_prob = 1 - enemy_heads_prob
    
    pvp_env = create_pvp_env(n_my_team, n_op_team)
    
    ### minus one to account for myself for later use
    n_my_team -= 1
        
    my_team_combs, op_team_combs = pvp_env[0], pvp_env[1]
    
    ### initiate my_team_probs
    my_team_probs = {}
    
    ### calculate probs for ALLY TEAM
    for i in range(len(my_team_combs)):
        comb_x_prob = my_team_combs[i] * ( ally_heads_prob**i * ally_tails_prob**(n_my_team-i) )
        my_team_probs[i] = comb_x_prob
        
    ### Add my contribution to my_team_probs
    my_team = list(my_team_probs.values())
    ### my_team_probs when I tails
    my_team_when_tails = [p*(1-my_heads_prob) for p in my_team]
    my_team_when_tails.append(0)
    ### my_team_probs when I heads
    my_team_when_heads = [p*my_heads_prob for p in my_team]
    my_team_when_heads.insert(0, 0)
    
    my_team = []
    for i in range(len(my_team_when_heads)):
        my_team.append(my_team_when_tails[i] + my_team_when_heads[i])
        
    for i in range(len(my_team_when_heads)):
        my_team_probs[i] = my_team[i]
    
    
    ### initiate op_team_probs
    op_team_probs = {}
    
    ### calculate probs for ENEMY TEAM
    for i in range(len(op_team_combs)):
        comb_x_prob = op_team_combs[i] * ( enemy_heads_prob**i * enemy_tails_prob**(n_op_team-i) )
        op_team_probs[i] = comb_x_prob
    
    
    
    result.append(my_team_probs)
    result.append(op_team_probs)
    
    return result

In [78]:
# def create_team_probs(n_my_team, n_op_team, my_heads_prob, ally_heads_prob, enemy_heads_prob):
team_probs = create_team_probs_dict(5, 5, my_heads_prob, ally_heads_prob,enemy_heads_prob)
team_probs

[{0: 0.019253060799999996,
  1: 0.11592717599999999,
  2: 0.279032688,
  3: 0.335608272,
  4: 0.201711384,
  5: 0.04846741920000001},
 {0: 0.020596297599999994,
  1: 0.12089131199999997,
  2: 0.283831776,
  3: 0.33319382399999997,
  4: 0.19557028800000004,
  5: 0.04591650240000002}]

In [79]:
team_probs[0].values()

dict_values([0.019253060799999996, 0.11592717599999999, 0.279032688, 0.335608272, 0.201711384, 0.04846741920000001])

In [80]:
# sum(team_probs[0].values())

In [81]:
### this function needs the result of create_team_probs as input


def create_multiplied_probs(team_probs):
    ### team_probs = result of function create_team_probs
    result = {}
    my_team_probs = list(team_probs[0].values())
    op_team_probs = list(team_probs[1].values())
    
    for i in range(len(my_team_probs)):
        result[i] = []
        for item in op_team_probs:
            result[i].append(my_team_probs[i]*item)
            
        
    return result

In [82]:
multiplied_probs = create_multiplied_probs(team_probs)

In [83]:
multiplied_probs

{0: [0.00039654176994769386,
  0.0023275277801277686,
  0.0054646304402999795,
  0.006415000951656497,
  0.0037653266455375104,
  0.000884033212430546],
 1: [0.0023876706168235767,
  0.014014588403094908,
  0.03290381625074457,
  0.03862621907696102,
  0.02267191119734669,
  0.005322970455029224],
 2: [0.005747040282175947,
  0.03373262774320664,
  0.07919834339709388,
  0.0929719683357189,
  0.054570503153574146,
  0.012812205088230454],
 3: [0.006912287847133745,
  0.04057212432013285,
  0.09525629188205106,
  0.11182260351371211,
  0.06563500641022235,
  0.015409958026747857],
 4: [0.004154507694171877,
  0.0243851538570958,
  0.057252100360137975,
  0.06720898737929241,
  0.0394487534617586,
  0.009261881247543324],
 5: [0.0009982493897471537,
  0.00585928989634199,
  0.0137565936696725,
  0.016149044742659022,
  0.009478787131560733,
  0.0022254543700186073]}

In [84]:
### function to determine which team has more HEADS
### this function needs the result of function create_multiplied_probs as input

def which_team_has_more(multiplied_probs):
    ### multiplied_probs = result of function create_multiplied_probs
    result = {}
    result['more_on_op'] = 0
    result['equal_on_both'] = 0
    result['more_on_my'] = 0
    
    
    ### iterate my team -> keys of given dict
    for item_1 in list(multiplied_probs.keys()):
        one_list = list(multiplied_probs.values())[item_1]
        # print('item_1: ', item_1)
        # print('-----------')
        # print('item_1: ', item_1)
        
        ### iterate one list of lists
        for i in range(len(one_list)):
            if i > item_1:
                result['more_on_op'] += one_list[i]
            if i == item_1:
                result['equal_on_both'] += one_list[i]
            elif i < item_1:
                result['more_on_my'] += one_list[i]
            else:
                pass
        
                
    return result

In [85]:
result = which_team_has_more(multiplied_probs)
result

{'more_on_op': 0.3690429582721709,
 'equal_on_both': 0.2471062849156258,
 'more_on_my': 0.3838507568122033}

In [86]:
def calculate_probs(n_my_team, n_op_team, my_heads_prob, ally_heads_prob, enemy_heads_prob):
    final_result = {}
    result_1 = create_team_probs_dict(n_my_team, n_op_team, my_heads_prob, ally_heads_prob, enemy_heads_prob)
    result_2 = create_multiplied_probs(result_1)
    result_3 = which_team_has_more(result_2)
    
    loss_prob = result_3['more_on_op'] + result_3['equal_on_both']/2
    win_prob = result_3['more_on_my'] + result_3['equal_on_both']/2
    
    final_result['loss_prob'] = loss_prob
    final_result['win_prob'] = win_prob
    
    return final_result

In [90]:
situation_1 = calculate_probs(5, 5, my_heads_prob, ally_heads_prob,enemy_heads_prob)
situation_1

{'loss_prob': 0.4925961007299838, 'win_prob': 0.5074038992700163}

In [104]:
# situation_2 = calculate_probs(5, 5, my_heads_prob, 0.54, 0.546)
# situation_2

# Ch.3 

In [92]:
wr_paul = situation_1['win_prob']
wr_paul

0.5074038992700163

In [97]:
net_games = 40

In [98]:
def amount_calculator(net_games, wr):
    return (net_games/(2*wr-1))

In [101]:
amount_calculator(net_games, wr_paul)

2701.2793219640907

In [102]:
amount_calculator(net_games, situation_1['loss_prob'])

-2701.279321964111